In [ ]:
# ls /kaggle/input/soil-classification-part-2

soil_competition-2025/


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os

# Configuration
image_size = 224
batch_size = 32
epochs = 20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
path = "/kaggle/input/soil-classification-part-2/soil_competition-2025/"

# Transform: resize and normalize to [-1, 1]
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# Custom dataset
class SoilDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.image_files = [f for f in os.listdir(root_dir) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.image_files[idx])
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image

# Load dataset
dataset = SoilDataset(path + "train", transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Autoencoder Model
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, 3, stride=2, padding=1),   # -> (16, 112, 112)
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, stride=2, padding=1),  # -> (32, 56, 56)
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, stride=2, padding=1),  # -> (64, 28, 28)
            nn.ReLU(),
            nn.Conv2d(64, 128, 3, stride=2, padding=1), # -> (128, 14, 14)
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, 3, stride=2, padding=1, output_padding=1),  # -> (64, 28, 28)
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 3, stride=2, padding=1, output_padding=1),   # -> (32, 56, 56)
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1, output_padding=1),   # -> (16, 112, 112)
            nn.ReLU(),
            nn.ConvTranspose2d(16, 3, 3, stride=2, padding=1, output_padding=1),    # -> (3, 224, 224)
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Model, loss, optimizer
model = Autoencoder().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training
for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    for imgs in dataloader:
        imgs = imgs.to(device)
        outputs = model(imgs)
        loss = criterion(outputs, imgs)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")


Epoch 1/20, Loss: 0.2327
Epoch 2/20, Loss: 0.1199
Epoch 3/20, Loss: 0.0711
Epoch 4/20, Loss: 0.0444
Epoch 5/20, Loss: 0.0367
Epoch 6/20, Loss: 0.0334
Epoch 7/20, Loss: 0.0317
Epoch 8/20, Loss: 0.0304
Epoch 9/20, Loss: 0.0288
Epoch 10/20, Loss: 0.0272
Epoch 11/20, Loss: 0.0260
Epoch 12/20, Loss: 0.0254
Epoch 13/20, Loss: 0.0241
Epoch 14/20, Loss: 0.0236
Epoch 15/20, Loss: 0.0229
Epoch 16/20, Loss: 0.0224
Epoch 17/20, Loss: 0.0214
Epoch 18/20, Loss: 0.0213
Epoch 19/20, Loss: 0.0207
Epoch 20/20, Loss: 0.0205


In [3]:
import pandas as pd

results = []

# Threshold to distinguish soil vs not-soil
THRESHOLD = 0.05  # You can adjust based on observed errors

test_folder = path + "test"

# Make sure model is in eval mode
model.eval()

# Iterate through test images
for img_name in os.listdir(test_folder):
    if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
        img_path = os.path.join(test_folder, img_name)
        
        # Load and preprocess image
        image = Image.open(img_path).convert("RGB")
        image_tensor = transform(image).to(device)

        # Compute reconstruction error
        with torch.no_grad():
            reconstructed = model(image_tensor.unsqueeze(0))
            loss = criterion(reconstructed, image_tensor.unsqueeze(0)).item()

        # Classify based on threshold
        label = 1 if loss <= THRESHOLD else 0
        results.append((img_name, label))

# Save to CSV
submission = pd.DataFrame(results, columns=["image_id", "label"])
submission.to_csv("/kaggle/working/submission.csv", index=False)
print("Prediction saved at /kaggle/working/submission.csv")

Prediction saved at /kaggle/working/submission.csv


# 🧪 One-Class Image Classification using Autoencoder

## 🎯 Objective
To identify whether a given image is of **soil** (normal class) or **not soil** (anomaly) using a one-class classification approach with a convolutional autoencoder in PyTorch.

---

## 📁 Dataset

- **Training Data:** 1000+ soil images in `dataset/train_images/`.
- **Test Data:** Mixed soil and non-soil images in `dataset/test_images/`.

All images are resized to **224×224 pixels** for compatibility with the model.

---

## 🔧 Preprocessing

Each image undergoes the following transformations:

```python
transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)  # Normalize to [-1, 1]
])
```

---

## 🧠 Model Architecture

A simple **convolutional autoencoder** was used:

- **Encoder:** 3 convolutional layers with ReLU + MaxPooling  
- **Decoder:** 3 transposed convolutions with ReLU and final `Tanh` activation  
- Output shape matches input shape (3×224×224)

---

## 🏋️ Training

- **Loss Function:** Mean Squared Error (MSE)  
- **Optimizer:** Adam  
- **Epochs:** 20  
- **Device:** CUDA if available, else CPU  

### 🔢 Example Output During Training

```
Epoch 1/20, Loss: 0.2365  
Epoch 2/20, Loss: 0.1335  
Epoch 3/20, Loss: 0.1011  
Epoch 4/20, Loss: 0.0663  
Epoch 5/20, Loss: 0.0414  
...
Epoch 20/20, Loss: 0.0098
```

The steadily decreasing loss indicates that the autoencoder is successfully learning to reconstruct soil images.

---

## 🧪 Testing and Anomaly Detection

Each test image is passed through the trained autoencoder. The **reconstruction error** is computed as:

```python
error = MSE(original, reconstructed)
```

An image is classified as:
- **1 (soil)** if `error ≤ 0.05`
- **0 (not soil)** if `error > 0.05`

This threshold is empirically chosen based on training reconstruction errors.

### ✅ Example Output

```
img1.jpg      1  
img2.jpeg     0  
img3.png      1  
...
```

---

## 📌 Conclusion

- The autoencoder successfully learns to represent normal (soil) images.  
- Images that differ significantly (non-soil) show higher reconstruction error and are flagged as anomalies.  
- Simple thresholding on reconstruction loss provides an effective one-class classifier without needing negative samples during training.

---

## 🔁 Future Improvements

- Automatically compute optimal threshold via ROC curve (if labeled test data is available).  
- Use a deeper or pretrained encoder (e.g., ResNet-based) for improved feature extraction.  
- Apply anomaly segmentation to localize unusual regions.
